# Analyzing Financial Services Consumer Complaints Data

## Objective

Using PostgreSQL to analyzing the data and discovering valuable insights

    Creating users, databases and tables,
    The IS NULL/IS NOT NULL clause,
    Views,
    Subqueries,
    Union/union all,
    Intersect/except,
    String concatenation, and
    Casting data types.

## Data Set

The data set contains two different CSV files of consumer complaint data from the CFPB:
    Bank_Account_or_Service_Complaints.csv
    Credit_Card_Complaints.csv

The files were downloaded from the [CFPB website](https://www.consumerfinance.gov/data-research/consumer-complaints/#download-the-data)

Both CSV files have identical fields. Below is each field, and the data type:

| Field Name                   | Data Type |
|------------------------------|-----------|
| complaint_id                 | text      |
| date_received                | date      |
| product                      | text      |
| sub_product                  | text      |
| issue                        | text      |
| sub_issue                    | text      |
| consumer_complaint_narrative | text      |
| company_public_response      | text      |
| company                      | text      |
| state                        | text      |
| zip_code                     | text      |
| tags                         | text      |
| consumer_consent_provided    | text      |
| submitted_via                | text      |
| date_sent                    | date      |
| company_response_to_consumer | text      |
| timely_response              | text      |
| consumer_disputed            | text      |



## Create New User, Database, and Tables using Pqsl Shell

Creating two new tables to store the csv data using the psql shell

    postgres=# CREATE ROLE oracle password 'welcome1' SUPERUSER CREATEDB CREATEROLE INHERIT LOGIN;
    CREATE ROLE
    postgres=# \du
                                   List of roles
     Role name |                         Attributes                         | Member of
    -----------+------------------------------------------------------------+-----------
     aig       | Superuser                                                  | {}
     oracle    | Superuser, Create role, Create DB                          | {}
     postgres  | Superuser, Create role, Create DB, Replication, Bypass RLS | {}
     sec       | Create DB                                                  | {}
     
     postgres=# SET ROLE oracle;
     SET
     

    postgres=# CREATE database consumer_complaints;
    CREATE DATABASE
    postgres=# \l
                                                     List of databases
            Name         |  Owner   | Encoding |          Collate          |           Ctype           |   Access privileges
    ---------------------+----------+----------+---------------------------+---------------------------+-----------------------
     bank_account        | postgres | UTF8     | Indonesian_Indonesia.1252 | Indonesian_Indonesia.1252 |
     consumer_complaints | oracle   | UTF8     | Indonesian_Indonesia.1252 | Indonesian_Indonesia.1252 |
     income              | postgres | UTF8     | Indonesian_Indonesia.1252 | Indonesian_Indonesia.1252 |
     postgres            | postgres | UTF8     | Indonesian_Indonesia.1252 | Indonesian_Indonesia.1252 |
     template0           | postgres | UTF8     | Indonesian_Indonesia.1252 | Indonesian_Indonesia.1252 | =c/postgres          +
                         |          |          |                           |                           | postgres=CTc/postgres
     template1           | postgres | UTF8     | Indonesian_Indonesia.1252 | Indonesian_Indonesia.1252 | =c/postgres          +
                         |          |          |                           |                           | postgres=CTc/postgres
    (6 rows)
    
    postgres=# \connect consumer_complaints
    consumer_complaints=#

### Creating tables for the data

    consumer_complaints=# CREATE TABLE credit_card_complaints (complaint_id text PRIMARY KEY,date_received date,product text,sub_product text,issue text,sub_issue text,consumer_complaint_narrative text,co
    mpany_public_response text,company text,state text,zip_code text,tags text,consumer_consent_provided text,submitted_via text,date_sent date,company_response_to_consumer text,timely_response text,consu
    mer_disputed text);
    CREATE TABLE
    consumer_complaints=# CREATE TABLE bank_account_complaints (complaint_id text PRIMARY KEY,date_received date,product text,sub_product text,issue text,sub_issue text,consumer_complaint_narrative text,c
    ompany_public_response text,company text,state text,zip_code text,tags text,consumer_consent_provided text,submitted_via text,date_sent date,company_response_to_consumer text,timely_response text,cons
    umer_disputed text);
    CREATE TABLE
    consumer_complaints=# \dt
                 List of relations
     Schema |          Name           | Type  | Owner
    --------+-------------------------+-------+--------
     public | bank_account_complaints | table | oracle
     public | credit_card_complaints  | table | oracle
    (2 rows)

## Loading Data into the Database

    consumer_complaints=# \COPY credit_card_complaints (date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zip_code,tags,consumer_consent_
    provided,submitted_via,date_sent,company_response_to_consumer,timely_response,consumer_disputed,complaint_id) FROM './Credit_Card_Complaints.csv' WITH CSV HEADER;
    COPY 87718
    
    consumer_complaints=# \COPY bank_account_complaints (date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zip_code,tags,consumer_consent
    _provided,submitted_via,date_sent,company_response_to_consumer,timely_response,consumer_disputed,complaint_id) FROM './Bank_Account_or_Service_Complaints.csv' WITH CSV HEADER;
    COPY 84811

## Querying Postgres Database

Run queries directly from psql shell can be difficult to understand the results, especially with many rows and columns.

Using the Pandas module, which will allow to store the results into a Dataframe object. This will make it easier to visualize and analyze the results. 

In [1]:
import pandas as pd

# psycopg2 lets us easily run commands against our db

import psycopg2
conn = psycopg2.connect("dbname=consumer_complaints user=oracle")
conn.autocommit = True
cur = conn.cursor()

def run_command(command):
    cur.execute(command)
    return cur.statusmessage

# sqlalchemy is needed to allow pandas to seemlessly connect to run queries

from sqlalchemy import create_engine
engine = create_engine('postgresql://oracle@localhost/consumer_complaints')

def run_query(query):
    return pd.read_sql_query(query,con=engine)

In [2]:
query = 'SELECT count(*) FROM credit_card_complaints;'
run_query(query)

,count
0,87718


In [3]:
query = 'SELECT count(*) FROM bank_account_complaints;'
run_query(query)

,count
0,84811


## Counting with Conditionals

See how many records in each table have null values for the consumer complaint narrative field, starting with the credit_card_complaints table.

In [4]:
query = '''
SELECT count(*) FROM credit_card_complaints 
WHERE consumer_complaint_narrative IS NOT NULL;
'''
run_query(query)

,count
0,17433


In [5]:
query = '''
SELECT count(*) FROM credit_card_complaints
WHERE consumer_complaint_narrative IS NULL;
'''
run_query(query)

,count
0,70285


Look at null values for the consumer_complaint_narrative column for the bank_account_complaints table.

In [6]:
query = '''
SELECT count(*) FROM bank_account_complaints
WHERE consumer_complaint_narrative IS NOT NULL;
'''
run_query(query)

,count
0,13860


In [7]:
query = '''
SELECT count(*) FROM bank_account_complaints
WHERE consumer_complaint_narrative IS NULL;
'''
run_query(query)

,count
0,70951


## Using Views

Create four views based on two dimensions: by product (eg credit card vs bank account), and whether they contain null values for consumer complaint narrative.

That four views will be called

    credit_card_w_complaints
    credit_card_wo_complaints
    bank_account_w_complaints
    bank_account_wo_complaints

In [8]:
command = '''
CREATE VIEW credit_card_w_complaints AS
    SELECT * FROM credit_card_complaints
    WHERE consumer_complaint_narrative IS NOT NULL;
'''
run_command(command)

'CREATE VIEW'

In [9]:
command = '''
CREATE VIEW credit_card_wo_complaints as
    SELECT * FROM credit_card_complaints
    WHERE consumer_complaint_narrative IS NULL;
'''
run_command(command)

'CREATE VIEW'

In [10]:
command = '''
CREATE VIEW bank_account_w_complaints AS
    SELECT * FROM bank_account_complaints
    WHERE consumer_complaint_narrative IS NOT NULL;
'''
run_command(command)

'CREATE VIEW'

In [11]:
command = '''
CREATE VIEW bank_account_wo_complaints AS
    SELECT * FROM bank_account_complaints
    WHERE consumer_complaint_narrative IS NULL;
'''
run_command(command)

'CREATE VIEW'

Take a quick look at credit_card_w_complaints, see that it works just like a table.

In [13]:
query = '''
SELECT * FROM credit_card_w_complaints LIMIT 5;
'''
run_query(query)

,complaint_id,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent,company_response_to_consumer,timely_response,consumer_disputed
0,1297939,2015-03-24,Credit card,None,Other,None,Received Capital One charge card offer XXXX. A...,None,Capital One,OH,440XX,None,Consent provided,Web,2015-03-30,Closed with explanation,Yes,No
1,1296693,2015-03-23,Credit card,None,Rewards,None,I 'm a longtime member of Charter One Bank/RBS...,None,"Citizens Financial Group, Inc.",MI,482XX,None,Consent provided,Web,2015-03-23,Closed with explanation,Yes,Yes
2,1295056,2015-03-23,Credit card,None,Other,None,I attempted to apply for a Discover Card Onlin...,None,Discover,MD,217XX,None,Consent provided,Web,2015-03-23,Closed with non-monetary relief,Yes,No
3,1296880,2015-03-23,Credit card,None,Late fee,None,XXXX Card services was bought out by Capital O...,None,Capital One,MI,488XX,None,Consent provided,Web,2015-03-23,Closed with monetary relief,Yes,No
4,1296890,2015-03-23,Credit card,None,Billing disputes,None,I was reported late by Discover Card to the re...,None,Discover,OK,741XX,Servicemember,Consent provided,Web,2015-03-23,Closed with explanation,Yes,No


## Union/Union All

Create two new views that will essentially consolidate the four current views down to two by performing a union on each products’ views.

In [14]:
command = '''
CREATE VIEW with_complaints AS
    SELECT * from credit_card_w_complaints
    UNION ALL
    SELECT * from bank_account_w_complaints;
'''
run_command(command)

'CREATE VIEW'

In [15]:
command = '''
CREATE VIEW without_complaints AS
    SELECT * FROM credit_card_wo_complaints
    UNION ALL
    SELECT * FROM bank_account_wo_complaints;
'''
run_command(command)

'CREATE VIEW'

## Intersect/Except

Validate to make sure the new views were created correctly

In [17]:
query = 'SELECT count(*) FROM credit_card_wo_complaints;'
run_query(query)

,count
0,70285


In [18]:
query = '''
SELECT count(*)
FROM (SELECT * FROM without_complaints
      INTERSECT
      SELECT * FROM credit_card_wo_complaints) ppg;
'''
run_query(query)

,count
0,70285


In [19]:
query = '''
SELECT count(*)
FROM (SELECT * FROM without_complaints
      EXCEPT
      SELECT * FROM credit_card_wo_complaints) ppg;
'''
run_query(query)

,count
0,70951


## String Concatenation

String concatenation is simply combining two or more strings (text values) together into a single string.

In [20]:
query = '''
SELECT complaint_id, product, company, zip_code,
       complaint_id || '-' || product || '-' || company || '-' || zip_code AS concat
FROM credit_card_complaints 
LIMIT 10
'''
run_query(query)

,complaint_id,product,company,zip_code,concat
0,469026,Credit card,Citibank,45247,469026-Credit card-Citibank-45247
1,469131,Credit card,Synchrony Financial,98548,469131-Credit card-Synchrony Financial-98548
2,479990,Credit card,Amex,78232,479990-Credit card-Amex-78232
3,475777,Credit card,Capital One,32226,475777-Credit card-Capital One-32226
4,469473,Credit card,Citibank,53066,469473-Credit card-Citibank-53066
5,470828,Credit card,Wells Fargo & Company,89108,470828-Credit card-Wells Fargo & Company-89108
6,470852,Credit card,Citibank,78249,470852-Credit card-Citibank-78249
7,479338,Credit card,JPMorgan Chase & Co.,19809,479338-Credit card-JPMorgan Chase & Co.-19809
8,480935,Credit card,Citibank,07018,480935-Credit card-Citibank-07018
9,469738,Credit card,Wells Fargo & Company,95409,469738-Credit card-Wells Fargo & Company-95409


## Subqueries

Subqueries help to simplify otherwise complex queries and allow for more flexibility

In [21]:
query = '''
SELECT ccd.complaint_id, ccd.product, ccd.company, ccd.zip_code
FROM (SELECT complaint_id, product, company, zip_code
      FROM credit_card_complaints
      WHERE zip_code = '91701') ccd 
LIMIT 10;
'''
run_query(query)

,complaint_id,product,company,zip_code
0,24857,Credit card,Barclays PLC,91701
1,33157,Credit card,Citibank,91701
2,12245,Credit card,Bank of America,91701
3,3151,Credit card,Barclays PLC,91701
4,352534,Credit card,Citibank,91701
5,1963836,Credit card,JPMorgan Chase & Co.,91701
6,2178015,Credit card,Discover,91701
7,2234754,Credit card,Discover,91701
8,2235915,Credit card,Discover,91701


In [22]:
query = '''
SELECT company, state, zip_code, count(complaint_id) AS complaint_count
FROM credit_card_complaints
WHERE company = 'Citibank'
 AND state IS NOT NULL
GROUP BY company, state, zip_code
ORDER BY 4 DESC
LIMIT 10;
'''
run_query(query)

,company,state,zip_code,complaint_count
0,Citibank,NY,100XX,80
1,Citibank,TX,750XX,67
2,Citibank,CA,945XX,58
3,Citibank,NY,112XX,52
4,Citibank,CA,900XX,50
5,Citibank,GA,300XX,46
6,Citibank,NJ,070XX,44
7,Citibank,IL,606XX,44
8,Citibank,FL,331XX,39
9,Citibank,CA,926XX,38


### See which companies receive the most complaints, and where the most complaints are generated from

In [23]:
query = '''
SELECT ens.company, ens.state, ens.zip_code, ens.complaint_count
FROM (select company, state, zip_code, count(complaint_id) AS complaint_count
      FROM credit_card_complaints
      WHERE state IS NOT NULL
      GROUP BY company, state, zip_code) ens
INNER JOIN
   (SELECT ppx.company, max(ppx.complaint_count) AS complaint_count
    FROM (SELECT ppt.company, ppt.state, max(ppt.complaint_count) AS complaint_count
          FROM (SELECT company, state, zip_code, count(complaint_id) AS complaint_count
                FROM credit_card_complaints
                WHERE company = 'Citibank' 
                 AND state IS NOT NULL
                GROUP BY company, state, zip_code
                ORDER BY 4 DESC) ppt
          GROUP BY ppt.company, ppt.state
          ORDER BY 3 DESC) ppx
    GROUP BY ppx.company) apx
ON apx.company = ens.company
 AND apx.complaint_count = ens.complaint_count
ORDER BY 4 DESC;
'''
run_query(query)

,company,state,zip_code,complaint_count
0,Citibank,NY,100XX,80


In [24]:
query = '''
SELECT ens.company, ens.state, ens.zip_code, ens.complaint_count
FROM (SELECT company, state, zip_code, count(complaint_id) AS complaint_count
      FROM credit_card_complaints
      WHERE state IS NOT NULL
      GROUP BY company, state, zip_code) ens
INNER JOIN
   (select ppx.company, max(ppx.complaint_count) AS complaint_count
    FROM (select ppt.company, ppt.state, max(ppt.complaint_count) AS complaint_count
          FROM (select company, state, zip_code, count(complaint_id) AS complaint_count
                FROM credit_card_complaints
                WHERE state IS NOT NULL
                GROUP BY company, state, zip_code
                ORDER BY 4 DESC) ppt
          GROUP BY ppt.company, ppt.state
          ORDER BY 3 DESC) ppx
    GROUP BY ppx.company) apx
ON apx.company = ens.company
AND apx.complaint_count = ens.complaint_count
ORDER BY 4 DESC
LIMIT 10
'''
run_query(query)

,company,state,zip_code,complaint_count
0,Citibank,NY,100XX,80
1,JPMorgan Chase & Co.,NY,100XX,44
2,Amex,NY,100XX,43
3,Bank of America,NY,10024,34
4,Capital One,NY,112XX,30
5,Capital One,GA,300XX,30
6,Synchrony Financial,NY,112XX,29
7,Barclays PLC,FL,337XX,28
8,Discover,MN,551XX,19
9,U.S. Bancorp,DC,200XX,19


## Casting Data Types

Run a query on the bank_account_complaints table to return the complaint_id as a float data type.

In [25]:
query = '''
SELECT CAST(complaint_id AS float) AS complaint_id
FROM bank_account_complaints LIMIT 10;'''
run_query(query)

,complaint_id
0,468889.0
1,468879.0
2,468949.0
3,468981.0
4,469185.0
5,475273.0
6,469309.0
7,469414.0
8,469446.0
9,469447.0


Test the query that had define the new view. 

In [27]:
query = '''
SELECT CAST(complaint_id AS int) AS complaint_id,
       date_received, product, sub_product, issue, company,
       state, zip_code, submitted_via, date_sent, company_response_to_consumer,
       timely_response, consumer_disputed
FROM bank_account_complaints 
WHERE state = 'CA'  
    AND consumer_disputed = 'No' 
    AND company = 'Wells Fargo & Company'
LIMIT 5;
'''
run_query(query)

,complaint_id,date_received,product,sub_product,issue,company,state,zip_code,submitted_via,date_sent,company_response_to_consumer,timely_response,consumer_disputed
0,468889,2013-07-29,Bank account or service,Checking account,Using a debit or ATM card,Wells Fargo & Company,CA,95992,Web,2013-07-31,Closed with explanation,Yes,No
1,469185,2013-07-29,Bank account or service,Other bank product/service,Deposits and withdrawals,Wells Fargo & Company,CA,96088,Web,2013-08-01,Closed with explanation,Yes,No
2,469446,2013-07-29,Bank account or service,Checking account,Deposits and withdrawals,Wells Fargo & Company,CA,92277,Web,2013-08-05,Closed with explanation,Yes,No
3,471075,2013-07-31,Bank account or service,Savings account,"Account opening, closing, or management",Wells Fargo & Company,CA,91214,Web,2013-08-02,Closed with explanation,Yes,No
4,463525,2013-07-22,Bank account or service,Other bank product/service,"Account opening, closing, or management",Wells Fargo & Company,CA,95376,Phone,2013-07-30,Closed with explanation,Yes,No


Create the new view named wells_complaints_v.

In [28]:
command = '''
CREATE VIEW wells_complaints_v AS (
    SELECT CAST(complaint_id AS int) AS complaint_id,
           date_received, product, sub_product, issue, company,
           state, zip_code, submitted_via, date_sent, company_response_to_consumer,
           timely_response, consumer_disputed
    FROM bank_account_complaints 
    WHERE state = 'CA'  
         AND consumer_disputed = 'No' 
         AND company = 'Wells Fargo & Company')
'''
run_command(command)

'CREATE VIEW'